In [1]:
import pandas as pd
import numpy as np
import math
from pandas.tseries.offsets import DateOffset
from datetime import timedelta

In [2]:
df=pd.read_csv("Grocery_Inventory_and_Sales_Dataset.csv")
df.head()

,Product_ID,Product_Name,Catagory,Supplier_ID,Supplier_Name,Stock_Quantity,Reorder_Level,Reorder_Quantity,Unit_Price,Date_Received,Last_Order_Date,Expiration_Date,Warehouse_Location,Sales_Volume,Inventory_Turnover_Rate,Status
0,29-205-1132,Sushi Rice,Grains & Pulses,38-037-1699,Jaxnation,22,72,70,$4.50,8/16/2024,6/29/2024,9/19/2024,48 Del Sol Trail,32,19,Discontinued
1,40-681-9981,Arabica Coffee,Beverages,54-470-2479,Feedmix,45,77,2,$20.00,11/1/2024,5/29/2024,5/8/2024,36 3rd Place,85,1,Discontinued
2,06-955-3428,Black Rice,Grains & Pulses,54-031-2945,Vinder,30,38,83,$6.00,8/3/2024,6/10/2024,9/22/2024,3296 Walton Court,31,34,Backordered
3,71-594-6552,Long Grain Rice,Grains & Pulses,63-492-7603,Brightbean,12,59,62,$1.50,12/8/2024,2/19/2025,4/17/2024,3 Westerfield Crossing,95,99,Active
4,57-437-1828,Plum,Fruits & Vegetables,54-226-4308,Topicstorm,37,30,74,$4.00,7/3/2024,10/11/2024,10/5/2024,15068 Scoville Court,62,25,Backordered


In [3]:
df.isnull().sum()

Product_ID                 0
Product_Name               0
Catagory                   1
Supplier_ID                0
Supplier_Name              0
Stock_Quantity             0
Reorder_Level              0
Reorder_Quantity           0
Unit_Price                 0
Date_Received              0
Last_Order_Date            0
Expiration_Date            0
Warehouse_Location         0
Sales_Volume               0
Inventory_Turnover_Rate    0
Status                     0
dtype: int64

In [4]:
df=df.dropna()

In [5]:
df.isnull().sum()

Product_ID                 0
Product_Name               0
Catagory                   0
Supplier_ID                0
Supplier_Name              0
Stock_Quantity             0
Reorder_Level              0
Reorder_Quantity           0
Unit_Price                 0
Date_Received              0
Last_Order_Date            0
Expiration_Date            0
Warehouse_Location         0
Sales_Volume               0
Inventory_Turnover_Rate    0
Status                     0
dtype: int64

In [6]:
#del unwanted col
df = df.drop(['Last_Order_Date', 'Warehouse_Location','Sales_Volume','Inventory_Turnover_Rate','Status' ], axis=1)
len(df)

989

In [7]:
df.head()

,Product_ID,Product_Name,Catagory,Supplier_ID,Supplier_Name,Stock_Quantity,Reorder_Level,Reorder_Quantity,Unit_Price,Date_Received,Expiration_Date
0,29-205-1132,Sushi Rice,Grains & Pulses,38-037-1699,Jaxnation,22,72,70,$4.50,8/16/2024,9/19/2024
1,40-681-9981,Arabica Coffee,Beverages,54-470-2479,Feedmix,45,77,2,$20.00,11/1/2024,5/8/2024
2,06-955-3428,Black Rice,Grains & Pulses,54-031-2945,Vinder,30,38,83,$6.00,8/3/2024,9/22/2024
3,71-594-6552,Long Grain Rice,Grains & Pulses,63-492-7603,Brightbean,12,59,62,$1.50,12/8/2024,4/17/2024
4,57-437-1828,Plum,Fruits & Vegetables,54-226-4308,Topicstorm,37,30,74,$4.00,7/3/2024,10/5/2024


In [8]:
'''
#swap
cols = list(df.columns)
i = cols.index('Date_Received')
j = cols.index('Expiration_Date')
cols[i], cols[j] = cols[j], cols[i]
df.columns = cols
df.head()
'''


"\n#swap\ncols = list(df.columns)\ni = cols.index('Date_Received')\nj = cols.index('Expiration_Date')\ncols[i], cols[j] = cols[j], cols[i]\ndf.columns = cols\ndf.head()\n"

In [9]:
df['Expiration_Date'] = pd.to_datetime(df['Expiration_Date'], format='%m/%d/%Y',errors='coerce')
df['Date_Received'] = pd.to_datetime(df['Date_Received'], format='%m/%d/%Y',errors='coerce')

# Calculate shelf life in days
df['Shelf_Life_(Days)'] = (df['Expiration_Date'] - df['Date_Received']).dt.days

# View a few rows
print(df[['Date_Received', 'Expiration_Date', 'Shelf_Life_(Days)']].head())
len(df)

  Date_Received Expiration_Date  Shelf_Life_(Days)
0    2024-08-16      2024-09-19                 34
1    2024-11-01      2024-05-08               -177
2    2024-08-03      2024-09-22                 50
3    2024-12-08      2024-04-17               -235
4    2024-07-03      2024-10-05                 94


989

In [10]:
#swap exp n recieved date of products with shelflife <0
'''df = df[df['Shelf_Life_(Days)'] >= 0]'''
# Swap values only where Shelf_Life_(Days) < 0
mask = df['Shelf_Life_(Days)'] < 0

# Use .loc to safely swap the values conditionally
df.loc[mask, ['Date_Received', 'Expiration_Date']] = df.loc[mask, ['Expiration_Date', 'Date_Received']].values

df['Shelf_Life_(Days)'] = (df['Expiration_Date'] - df['Date_Received']).dt.days

print(df[['Date_Received', 'Expiration_Date', 'Shelf_Life_(Days)']].head())
len(df)

  Date_Received Expiration_Date  Shelf_Life_(Days)
0    2024-08-16      2024-09-19                 34
1    2024-05-08      2024-11-01                177
2    2024-08-03      2024-09-22                 50
3    2024-04-17      2024-12-08                235
4    2024-07-03      2024-10-05                 94


989

In [11]:
df['Shelf_Life_(Years)'] = (df['Shelf_Life_(Days)']/365).round(2)
print(df[['Date_Received', 'Expiration_Date', 'Shelf_Life_(Days)','Shelf_Life_(Years)']].head())
len(df)

  Date_Received Expiration_Date  Shelf_Life_(Days)  Shelf_Life_(Years)
0    2024-08-16      2024-09-19                 34                0.09
1    2024-05-08      2024-11-01                177                0.48
2    2024-08-03      2024-09-22                 50                0.14
3    2024-04-17      2024-12-08                235                0.64
4    2024-07-03      2024-10-05                 94                0.26


989

In [12]:
max_years = math.ceil(df['Shelf_Life_(Years)'].max())
bins = list(range(0, max_years + 1))
labels = [f'{i}-{i+1} years' for i in range(0, max_years)]
df['Shelf_Life_Bin'] = pd.cut(df['Shelf_Life_(Years)'], bins=bins, labels=labels, right=False)
shelf_life_distribution = df['Shelf_Life_Bin'].value_counts().sort_index()
print(shelf_life_distribution)
len(df)

Shelf_Life_Bin
0-1 years    988
Name: count, dtype: int64


989

In [13]:
len(df)

989

In [14]:
df=df.copy()
df.reset_index(drop=True,inplace=True)
df.head()

,Product_ID,Product_Name,Catagory,Supplier_ID,Supplier_Name,Stock_Quantity,Reorder_Level,Reorder_Quantity,Unit_Price,Date_Received,Expiration_Date,Shelf_Life_(Days),Shelf_Life_(Years),Shelf_Life_Bin
0,29-205-1132,Sushi Rice,Grains & Pulses,38-037-1699,Jaxnation,22,72,70,$4.50,2024-08-16,2024-09-19,34,0.09,0-1 years
1,40-681-9981,Arabica Coffee,Beverages,54-470-2479,Feedmix,45,77,2,$20.00,2024-05-08,2024-11-01,177,0.48,0-1 years
2,06-955-3428,Black Rice,Grains & Pulses,54-031-2945,Vinder,30,38,83,$6.00,2024-08-03,2024-09-22,50,0.14,0-1 years
3,71-594-6552,Long Grain Rice,Grains & Pulses,63-492-7603,Brightbean,12,59,62,$1.50,2024-04-17,2024-12-08,235,0.64,0-1 years
4,57-437-1828,Plum,Fruits & Vegetables,54-226-4308,Topicstorm,37,30,74,$4.00,2024-07-03,2024-10-05,94,0.26,0-1 years


In [15]:
len(df)

989

In [16]:
#exp
#curr_date=df['Date_Received'].max()
curr_date='2025-01-01'
expired_items = df[df['Expiration_Date'] < curr_date]
not_exp=len(df)-expired_items.shape[0]
print(not_exp)

265


In [17]:
len(df[df['Date_Received']<curr_date])

969

In [18]:
#setting curr date as 2025-02-09
''' tables=expiered, discount, donation
create status in i_table

'''

' tables=expiered, discount, donation\ncreate status in i_table\n\n'

In [19]:
condition = df['Date_Received'] < pd.Timestamp('2024-11-01')

# Apply 7-month shift only to matching rows
df.loc[condition, 'Date_Received'] = df.loc[condition, 'Date_Received'] + pd.DateOffset(months=7)
df.loc[condition, 'Expiration_Date'] = df.loc[condition, 'Expiration_Date'] + pd.DateOffset(months=7)

# View result
print(df[['Date_Received', 'Expiration_Date']].head())

  Date_Received Expiration_Date
0    2025-03-16      2025-04-19
1    2024-12-08      2025-06-01
2    2025-03-03      2025-04-22
3    2024-11-17      2025-07-08
4    2025-02-03      2025-05-05


In [20]:
df['Date_Received'].max()

Timestamp('2025-05-31 00:00:00')

In [21]:
df['Date_Received'].min()

Timestamp('2024-09-25 00:00:00')

In [22]:
#abhi bas moths aage karke dekh rhe hai
curr_date=df['Date_Received'].max()
expired_items = df[df['Expiration_Date'] < curr_date]
not_exp=len(df)-expired_items.shape[0]
print(not_exp)

425


In [23]:

# Calculate shelf life in days
df['Shelf_Life_(Days)'] = (df['Expiration_Date'] - df['Date_Received']).dt.days

# View a few rows
print(df[['Date_Received', 'Expiration_Date', 'Shelf_Life_(Days)']].head())


  Date_Received Expiration_Date  Shelf_Life_(Days)
0    2025-03-16      2025-04-19                 34
1    2024-12-08      2025-06-01                175
2    2025-03-03      2025-04-22                 50
3    2024-11-17      2025-07-08                233
4    2025-02-03      2025-05-05                 91


In [24]:
#making col for donation and discount dates in df

In [25]:
df['Expiration_Date'] = pd.to_datetime(df['Expiration_Date'], format='%m/%d/%Y')

# Apply conditional logic for discount days:
# 20% of shelf life if bin is '0-1 years', otherwise 10%
discount_fraction = np.where(df['Shelf_Life_Bin'] == '0-1 years', 0.2, 0.1)

# Calculate Discount_Date accordingly
df['Discount_Date'] = df['Expiration_Date'] - pd.to_timedelta(df['Shelf_Life_(Days)'] * discount_fraction, unit='D')

# View result
print(df[['Expiration_Date', 'Shelf_Life_(Days)', 'Shelf_Life_Bin', 'Discount_Date', 'Date_Received']].head())

  Expiration_Date  Shelf_Life_(Days) Shelf_Life_Bin       Discount_Date  \
0      2025-04-19                 34      0-1 years 2025-04-12 04:48:00   
1      2025-06-01                175      0-1 years 2025-04-27 00:00:00   
2      2025-04-22                 50      0-1 years 2025-04-12 00:00:00   
3      2025-07-08                233      0-1 years 2025-05-22 09:36:00   
4      2025-05-05                 91      0-1 years 2025-04-16 19:12:00   

  Date_Received  
0    2025-03-16  
1    2024-12-08  
2    2025-03-03  
3    2024-11-17  
4    2025-02-03  


In [26]:
df['Expiration_Date'] = pd.to_datetime(df['Expiration_Date'], format='%m/%d/%Y')

# Set donation fraction based on shelf life bin
donation_fraction = np.where(df['Shelf_Life_Bin'] == '0-1 years', 0.1, 0.05)

# Calculate Donation_Date accordingly
df['Donation_Date'] = df['Expiration_Date'] - pd.to_timedelta(df['Shelf_Life_(Days)'] * donation_fraction, unit='D')

# View result
print(df[['Expiration_Date', 'Shelf_Life_(Days)', 'Shelf_Life_Bin', 'Donation_Date', 'Date_Received']].head())

  Expiration_Date  Shelf_Life_(Days) Shelf_Life_Bin       Donation_Date  \
0      2025-04-19                 34      0-1 years 2025-04-15 14:24:00   
1      2025-06-01                175      0-1 years 2025-05-14 12:00:00   
2      2025-04-22                 50      0-1 years 2025-04-17 00:00:00   
3      2025-07-08                233      0-1 years 2025-06-14 16:48:00   
4      2025-05-05                 91      0-1 years 2025-04-25 21:36:00   

  Date_Received  
0    2025-03-16  
1    2024-12-08  
2    2025-03-03  
3    2024-11-17  
4    2025-02-03  


In [27]:
# Inventory_table
inventory_table = df[['Product_ID', 'Product_Name', 'Catagory', 'Supplier_ID', 'Supplier_Name', 'Stock_Quantity', 'Reorder_Level', 'Reorder_Quantity','Unit_Price','Date_Received','Expiration_Date']]

In [28]:
df.head()

,Product_ID,Product_Name,Catagory,Supplier_ID,Supplier_Name,Stock_Quantity,Reorder_Level,Reorder_Quantity,Unit_Price,Date_Received,Expiration_Date,Shelf_Life_(Days),Shelf_Life_(Years),Shelf_Life_Bin,Discount_Date,Donation_Date
0,29-205-1132,Sushi Rice,Grains & Pulses,38-037-1699,Jaxnation,22,72,70,$4.50,2025-03-16,2025-04-19,34,0.09,0-1 years,2025-04-12 04:48:00,2025-04-15 14:24:00
1,40-681-9981,Arabica Coffee,Beverages,54-470-2479,Feedmix,45,77,2,$20.00,2024-12-08,2025-06-01,175,0.48,0-1 years,2025-04-27 00:00:00,2025-05-14 12:00:00
2,06-955-3428,Black Rice,Grains & Pulses,54-031-2945,Vinder,30,38,83,$6.00,2025-03-03,2025-04-22,50,0.14,0-1 years,2025-04-12 00:00:00,2025-04-17 00:00:00
3,71-594-6552,Long Grain Rice,Grains & Pulses,63-492-7603,Brightbean,12,59,62,$1.50,2024-11-17,2025-07-08,233,0.64,0-1 years,2025-05-22 09:36:00,2025-06-14 16:48:00
4,57-437-1828,Plum,Fruits & Vegetables,54-226-4308,Topicstorm,37,30,74,$4.00,2025-02-03,2025-05-05,91,0.26,0-1 years,2025-04-16 19:12:00,2025-04-25 21:36:00


In [29]:
condition = (df['Discount_Date'] <= curr_date) & (curr_date < df['Donation_Date'])
selected_columns = ['Product_ID', 'Product_Name', 'Catagory', 'Supplier_ID', 'Supplier_Name', 'Stock_Quantity', 'Reorder_Level', 'Reorder_Quantity','Unit_Price','Date_Received','Expiration_Date','Discount_Date']

# Create discount table
discount_table = df.loc[condition, selected_columns]

# View result
discount_table.head()


,Product_ID,Product_Name,Catagory,Supplier_ID,Supplier_Name,Stock_Quantity,Reorder_Level,Reorder_Quantity,Unit_Price,Date_Received,Expiration_Date,Discount_Date
3,71-594-6552,Long Grain Rice,Grains & Pulses,63-492-7603,Brightbean,12,59,62,$1.50,2024-11-17,2025-07-08,2025-05-22 09:36:00
9,46-452-9419,Egg (Duck),Dairy,67-137-4215,Wordify,43,10,15,$1.00,2025-02-08,2025-06-18,2025-05-23 00:00:00
40,11-316-8405,Mango,Fruits & Vegetables,50-918-1373,Twitternation,24,55,51,$5.00,2024-11-26,2025-06-25,2025-05-13 19:12:00
54,36-918-2937,Green Coffee,Beverages,53-844-0478,Kwinu,62,89,18,$12.00,2024-10-12,2025-06-28,2025-05-07 04:48:00
66,20-225-3930,White Tea,Beverages,73-069-2753,Skipstorm,99,87,10,$25.00,2024-10-17,2025-07-08,2025-05-16 04:48:00


In [30]:
condition = (df['Donation_Date'] <= curr_date) & (df['Expiration_Date'] > curr_date)
selected_columns = ['Product_ID', 'Product_Name', 'Catagory', 'Supplier_ID', 'Supplier_Name', 'Stock_Quantity', 'Reorder_Level', 'Reorder_Quantity','Unit_Price','Date_Received','Expiration_Date','Donation_Date']

# Create discount table
donation_table = df.loc[condition, selected_columns]

# View result
donation_table.head()


,Product_ID,Product_Name,Catagory,Supplier_ID,Supplier_Name,Stock_Quantity,Reorder_Level,Reorder_Quantity,Unit_Price,Date_Received,Expiration_Date,Donation_Date
1,40-681-9981,Arabica Coffee,Beverages,54-470-2479,Feedmix,45,77,2,$20.00,2024-12-08,2025-06-01,2025-05-14 12:00:00
16,55-936-2406,Bread Flour,Grains & Pulses,29-689-5014,Mynte,27,93,88,$1.50,2024-11-05,2025-06-16,2025-05-24 16:48:00
26,02-508-3777,Egg (Quail),Dairy,69-933-2582,Realpoint,97,88,81,$0.80,2025-01-26,2025-06-05,2025-05-23 00:00:00
33,79-136-9840,Herbal Tea,Beverages,07-809-0926,Pixonyx,77,45,75,$30.00,2025-01-23,2025-06-05,2025-05-22 16:48:00
45,90-230-9767,Long Grain Rice,Grains & Pulses,70-515-9048,Voonyx,67,88,32,$1.50,2024-11-06,2025-06-17,2025-05-25 16:48:00


In [31]:
#Expired_table
condition =  (df['Expiration_Date'] <= curr_date)
selected_columns = ['Product_ID', 'Product_Name', 'Catagory', 'Supplier_ID', 'Supplier_Name', 'Stock_Quantity', 'Reorder_Level', 'Reorder_Quantity','Unit_Price','Date_Received','Expiration_Date']

# Create discount table
expired_table = df.loc[condition, selected_columns]

# View result
expired_table.head()


,Product_ID,Product_Name,Catagory,Supplier_ID,Supplier_Name,Stock_Quantity,Reorder_Level,Reorder_Quantity,Unit_Price,Date_Received,Expiration_Date
0,29-205-1132,Sushi Rice,Grains & Pulses,38-037-1699,Jaxnation,22,72,70,$4.50,2025-03-16,2025-04-19
2,06-955-3428,Black Rice,Grains & Pulses,54-031-2945,Vinder,30,38,83,$6.00,2025-03-03,2025-04-22
4,57-437-1828,Plum,Fruits & Vegetables,54-226-4308,Topicstorm,37,30,74,$4.00,2025-02-03,2025-05-05
6,71-516-1996,Corn Oil,Oils & Fats,04-391-7610,Tagfeed,96,52,16,$2.50,2024-10-18,2025-01-20
7,39-629-5554,Egg (Goose),Dairy,67-679-4930,Muxo,44,90,17,$2.50,2025-02-03,2025-02-05


In [32]:
#fresh
condition =  (df['Discount_Date'] > curr_date)
selected_columns = ['Product_ID', 'Product_Name', 'Catagory', 'Supplier_ID', 'Supplier_Name', 'Stock_Quantity', 'Reorder_Level', 'Reorder_Quantity','Unit_Price','Date_Received','Expiration_Date']

# Create discount table
fresh_table = df.loc[condition, selected_columns]

# View resul
fresh_table.head()


,Product_ID,Product_Name,Catagory,Supplier_ID,Supplier_Name,Stock_Quantity,Reorder_Level,Reorder_Quantity,Unit_Price,Date_Received,Expiration_Date
5,21-120-6238,All-Purpose Flour,Grains & Pulses,86-292-4587,Dabjam,55,33,14,$1.75,2025-04-05,2025-07-03
12,75-094-1179,Rye Bread,Bakery,57-615-0729,Vitz,26,79,31,$3.00,2025-02-18,2025-07-13
19,76-854-0095,White Sugar,Grains & Pulses,38-883-3382,Mydeo,65,61,82,$2.00,2024-12-31,2025-07-28
24,68-418-6724,Parmesan Cheese,Dairy,75-465-6257,Feedspan,63,4,85,$12.00,2025-04-30,2025-07-21
28,31-211-5803,Oatmeal Biscuit,Bakery,75-503-9347,Gigazoom,16,9,32,$5.00,2025-02-16,2025-09-07


In [33]:
len(fresh_table)+len(expired_table)+len(discount_table)+len(donation_table)

989

In [34]:
len(df)

989